### Web Scrapping

In [ ]:
# !pip install --upgrade pip
# !pip install beautifulsoup4 --upgrade
# pip install lxml
# pip install html5lib

In [1]:
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup
from json2html import *
import re
import requests
from urllib.request import Request, urlopen
from urllib import parse
import glob

#### External HTML File From Browser Element Section 

In [166]:
lang = "indonesian"  # indonesian, turkish, russian, english, urdu, hindi, bangla
surah = "al mumtahana"  # al faatiha, al baqara, aal i imraan ...
folder = f"{surah}-60"  # al faatiha-1, al baqara-2, aal i imraan-3 ...
file = f"{surah.lower()}_{lang.lower()}"

In [167]:
HTMLFile = open(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Arabic/Quaran/Web Scrapping/Data/{folder.lower().capitalize()}/{file.lower()}.html", "r") # ayahs-block

content = HTMLFile.read()

soup = BeautifulSoup(content, 'lxml')

ayah_word = soup.findAll('span', attrs={"class":"single-word"})

word_list = []
for word in ayah_word:
    var1 = word.find('span', attrs={"class":"word-arabic word-arabic-uthmani-hafs"})
    #try:
    #    #arabic = var1.get_text()
    #    arabic = var1.string
    #except:
    #    arabic = ""
    #arabic = var1.get_text()
    arabic = var1.string
    var2 = word.find('span', attrs={"class":"word-transliteration"})
    transliterate = var2.string
    var3 = word.find('span', attrs={"class":"word-translation"})
    translate = var3.string
    word_list.append((arabic, transliterate, translate))

df_word_list = pd.DataFrame(word_list)
df_word_list.rename(columns={0:"arabic", 1:"transliterate", 2:f"translate_{lang.lower()}"}, inplace=True)
df_word_list

,arabic,transliterate,translate_indonesian
0,يَـٰٓأَيُّهَا,yaayyuha,wahai
1,ٱلَّذِينَ,alladhina,orang-orang yang
2,ءَامَنُواْ,amanu,beriman
3,لَا,la,janganlah
4,تَتَّخِذُواْ,tattakhidhu,kamu mengambil
...,...,...,...
343,يَئِسَ,ya-isa,berputus asa
344,ٱلۡكُفَّارُ,al-kufaru,orang-orang kafir
345,مِنۡ,min,dari
346,أَصۡحَٰبِ,ashabi,penghuni


In [168]:
df_word_list.to_excel(f"{surah.lower().capitalize()}_Arabic_Transliterate_Translate_Words_{lang.lower().capitalize()}.xlsx", index=False)

#### Merge Word All Translate

In [169]:
all_trans_file = glob.glob(f"{surah.lower().capitalize()}*")
all_trans_file

['Al mumtahana_Arabic_Transliterate_Translate_Words_Bangla.xlsx',
 'Al mumtahana_Arabic_Transliterate_Translate_Words_Hindi.xlsx',
 'Al mumtahana_Arabic_Transliterate_Translate_Words_Urdu.xlsx',
 'Al mumtahana_Arabic_Transliterate_Translate_Words_English.xlsx',
 'Al mumtahana_Arabic_Transliterate_Translate_Words_Russian.xlsx',
 'Al mumtahana_Arabic_Transliterate_Translate_Words_Turkish.xlsx',
 'Al mumtahana_Arabic_Transliterate_Translate_Words_Indonesian.xlsx']

In [170]:
df_master = pd.read_excel(all_trans_file[0])
for i in all_trans_file[1:]:
    df_var = pd.read_excel(f"{i}")
    #df_master = pd.merge(df_master, df_var, how="left", on=["arabic","transliterate"])
    df_master = pd.concat([df_master,df_var.iloc[:,2]], axis=1)
    #df_master = df_master.join(df_var, how="left", lsuffix=["arabic","transliterate"],rsuffix=["arabic","transliterate"])
    #df_master.drop_duplicates(inplace=True)
    df_master.reset_index(drop=True, inplace=True)
df_master = df_master[["arabic","transliterate","translate_english","translate_urdu","translate_hindi","translate_indonesian","translate_bangla","translate_turkish","translate_russian"]]
df_master

,arabic,transliterate,translate_english,translate_urdu,translate_hindi,translate_indonesian,translate_bangla,translate_turkish,translate_russian
0,يَـٰٓأَيُّهَا,yaayyuha,O you,اے,*,wahai,ওহে,ey,"О те,"
1,ٱلَّذِينَ,alladhina,who,لوگو,ऐ लोगो जो,orang-orang yang,যারা,kimseler,которые
2,ءَامَنُواْ,amanu,believe,جو ایمان لائے ہو,ईमान लाए हो,beriman,ঈমান এনেছ,inanan(lar),уверовали!
3,لَا,la,(Do) not,نہ,*,janganlah,না,edinmeyin,Не
4,تَتَّخِذُواْ,tattakhidhu,take,تم بناؤ,ना तुम बनाओ,kamu mengambil,তোমরা গ্রহণ করো,*,берите
...,...,...,...,...,...,...,...,...,...
343,يَئِسَ,ya-isa,despair,مایوس ہوئے,मायूस हुए,berputus asa,নিরাশ হয়েছে,umudu kestiği,отчаялись
344,ٱلۡكُفَّارُ,al-kufaru,the disbelievers,کافر,काफ़िर,orang-orang kafir,কাফিররা,kafirlerin,неверующие
345,مِنۡ,min,of,سے,*,dari,থেকে,halkından,из числа
346,أَصۡحَٰبِ,ashabi,(the) companions,والوں سے,*,penghuni,অধিবাসীদের,*,обитателей


In [171]:
df_master.to_excel(f"{surah.lower().capitalize()}_Arabic_Transliterate_Translate_Words_Languages_Concat.xlsx", index=False)

#### Merge Surah All Translate

In [ ]:
def add_info(df,surah_name, surah_num, index):
    for i in range(len(df)):
        df["surah latin"] = surah_name
        df["surah"] = surah_num
        df.loc[i,"index"] = i+index
    return df

In [ ]:
df_faatir = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Arabic/Quaran/Web Scrapping/Result/Faatir-35/Faatir_Arabic_Transliterate_Translate_Words_Languages_Concat.xlsx")
df_faatir

In [ ]:
df_faatir = add_info(df_faatir, "Fatir", 35, 56320)
df_faatir = df_faatir.iloc[:,[10,9,0,1,2,3,4,5,6,7,8,11]]
df_faatir

In [ ]:
df_yaseen = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Arabic/Quaran/Web Scrapping/Result/Yaseen-36/Yaseen_Arabic_Transliterate_Translate_Words_Languages_Concat.xlsx")
df_yaseen

In [ ]:
df_yaseen = add_info(df_yaseen, "Ya Seen", 36, 57102)
df_yaseen = df_yaseen.iloc[:,[10,9,0,1,2,3,4,5,6,7,8,11]]
df_yaseen

In [ ]:
df_saaffaat = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Arabic/Quaran/Web Scrapping/Result/As Saaffaat-37/As saaffaat_Arabic_Transliterate_Translate_Words_Languages_Concat.xlsx")
df_saaffaat

In [ ]:
df_saaffaat = add_info(df_saaffaat, "As-Saffat", 37, 57836)
df_saaffaat = df_saaffaat.iloc[:,[10,9,0,1,2,3,4,5,6,7,8,11]]
df_saaffaat

In [ ]:
df_saad = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Arabic/Quaran/Web Scrapping/Result/Saad-38/Saad_Arabic_Transliterate_Translate_Words_Languages_Concat.xlsx")
df_saad

In [ ]:
df_saad = add_info(df_saad, "Sad", 38, 58705)
df_saad = df_saad.iloc[:,[10,9,0,1,2,3,4,5,6,7,8,11]]
df_saad

In [ ]:
df_zumar = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Arabic/Quaran/Web Scrapping/Result/Az Zumar-39/Az zumar_Arabic_Transliterate_Translate_Words_Languages_Concat.xlsx")
df_zumar

In [ ]:
df_zumar = add_info(df_zumar, "Az-Zumar", 39, 59444)
df_zumar = df_zumar.iloc[:,[10,9,0,1,2,3,4,5,6,7,8,11]]
df_zumar

In [ ]:
df_surah_merge = pd.concat([df_faatir,df_yaseen,df_saaffaat,df_saad,df_zumar], axis=0)
df_surah_merge

In [ ]:
df_surah_merge.to_excel("Faatir_Yaseen_Saaffaat_Saad_Zumar_Surah_Translate_Concat.xlsx", index=False)